# EEEN30131 Power System Analysis: Week 06 - Examples

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides several examples covering frequency regulation and generation control, including general `python` code which can be used to solve the examples and create new ones.

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as such activities will better prepare you for the exams.

## List of contents

- [Two-bus system example](#Two-bus-system-example)
- [Power circle diagram](#Power-circle-diagram)
- [Series compensation](#Series-compensation)
- [Generators](#Generators)
- [Tap-changing transformers](#Tap-changing-transformers)

## Before we begin

Before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131 Week 6 - Primary voltage control 

This notebook builds on the materials that were presented in previous sessions. Therefore, make sure to review the asynchronous materials provided in blackboard for EEEN30131:
  - Week 1 - Nodal analysis 
  - Week 2 - Power Flow Formulation 
  - Week 3 - Newton Raphson
  - Week 4 - Frequency regulation
  - Week 5 - Interconnected systems

- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [1]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

The following libraries are required by the notebook:

In [2]:
try:
    import ipywidgets as widgets
except:
    import micropip
    await micropip.install('ipywidgets')
    import ipywidgets as widgets
from ipywidgets import interact
import cmath
import numpy
import matplotlib.pyplot as plt
import math

The notebook borrows several tools developed in previous weeks, so we need to import them here:

In [3]:
from Code.Wk1_EEEN30131 import get_Ybus
from Code.Wk2_EEEN30131 import get_Bus_Type, develop_PF_Equations
from Code.Wk3_EEEN30131 import Newtons_Method
from Code.Wk6_EEEN30131 import bfs, get_Parameters, get_power_circle, plot_circles, plot_vector, plot_phasor

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)

## Two-bus system example

Consider the system presented below.
- What can you do to adjust the voltage in bus 2?
- What happens if the load changes or becomes negative (e.g., distributed generation)?
- Try to keep the voltage at 0.95 pu as you change the loads, is that easy to do?

![Week02_2Bus.png](Figures/Week02_2Bus.png)

In [4]:
@interact
def VC_01(P = widgets.FloatSlider(min=-5,max=5,step=0.01, value=0.5,description='P2 pu'),
         Q = widgets.FloatSlider(min=-5,max=5,step=0.01,value=0.5,description='Q2 pu'),
         V = widgets.FloatSlider(min=0.95,max=1.05,step=0.01,value=1,description='V pu')):

    Connectivity = [
        [1, 2, complex(0, 0.1), 0]
    ]
    Load = [
        [2, complex(P, Q)]
    ]
    Generator = [
        {'Bus':1, 'V':V, '𝜃':0 }
    ]
    V_All, 𝜃_All, Threshold, Succes = bfs(Generator, Connectivity, Load)
    for xb, V, 𝜃 in zip(range(len(V_All)), V_All, 𝜃_All):
        print('V%d =  %7.4f ∠ %8.4f [pu][deg]'%(xb+1, V, 𝜃*180/math.pi))

interactive(children=(FloatSlider(value=0.5, description='P2 pu', max=5.0, min=-5.0, step=0.01), FloatSlider(v…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)

## Power circle diagram

Use the 2-bus example and plot both power circle diagrams. Assume the impedance of the line is 0.03+j0.33 $\Omega$. Once we **fix the voltage at the sending end**, the voltage at the receiving end will remain the same for any PQ **load** combinations (in bus 2) on the perimeter of the circle.

In [5]:
@interact
def PCR_02(P = widgets.FloatSlider(min=0,max=1,step=0.01, value=0.1,description='P2 pu'),
         Q = widgets.FloatSlider(min=-0.3,max=0.1,step=0.01,value=0.01,description='Q2 pu')):

    Connectivity = [
        [1, 2, complex(0.03, 0.33), 0]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 }
    ]
    Load = [
        [2, complex(0.1, 0.01)]
    ]
    V_Original, _, Threshold, Succes = bfs(Generator, Connectivity, Load)
    radius, centreR, centreS = get_power_circle(V_Original, Connectivity)
    Vr = V_Original[1]
    
    Axes = [0, 1, -0.3, 0.1]
    print('Adjust the load, especially Q2, to keep V2 at %.4f pu'%Vr)
    Load = [
        [2, complex(P, Q)]
    ]

    V_All, 𝜃_All, Threshold, Succes = bfs(Generator, Connectivity, Load)
    Results = get_Parameters(Connectivity, V_All, 𝜃_All, Succes)
    for xb, V, 𝜃 in zip(range(len(V_All)), V_All, 𝜃_All):
        print('  V%d =  %7.4f ∠ %8.4f [pu][deg]'%(xb+1, V, 𝜃*180/math.pi))
    
    y1 = (radius**2 - (P - centreR[0])**2)**0.5 + centreR[1]
    y2 = -(radius**2 - (P - centreR[0])**2)**0.5 + centreR[1]
    print('Consider Q2 = %.4f or Q2 = %.4f'%(y1, y2))
    PG = Results['Net_Power'][0].real/100
    QG = Results['Net_Power'][0].imag/100

    plot_circles(centreR, centreS, radius, [P, Q], [PG, QG], Axes)

interactive(children=(FloatSlider(value=0.1, description='P2 pu', max=1.0, step=0.01), FloatSlider(value=0.01,…

Based on the sending end power circle diagram, once we **fix the voltage at the receiving** end, the voltage at the sending end will remain the same for any PQ **generation** combinations (in bus 1) on the perimeter of the circle.

In [6]:
@interact
def PCR_01(PG = widgets.FloatSlider(min=0,max=1,step=0.01, value=0.1003, description='P1 pu'),
         QG = widgets.FloatSlider(min=0,max=0.1,step=0.001, value=0.0133, readout_format='.3f', description='Q1 pu')):

    Connectivity = [
        [1, 2, complex(0.03, 0.33), 0]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 }
    ]
    Load = [
        [2, complex(0.1, 0.01)]
    ]
    V_Original, _, Threshold, Succes = bfs(Generator, Connectivity, Load)
    radius, centreR, centreS = get_power_circle(V_Original, Connectivity)
    Vr = V_Original[1]
    Vs = V_Original[0]
    
    Load = [
        [2, complex(-PG, -QG)]
    ]
    Generator = [
        {'Bus':1, 'V':Vr, '𝜃':0 }
    ]
    Axes = [0, 1, -0.3, 0.1]

    print('Adjust generation, especially Q1, to keep V1 at %.4f pu'%Vs)
    V_All, 𝜃_All, Threshold, Succes = bfs(Generator, Connectivity, Load)
    Results = get_Parameters(Connectivity, V_All, 𝜃_All, Succes)
    print('  V%d =  %7.4f ∠ %8.4f [pu][deg]'%(1, V_All[1], 𝜃_All[0]*180/math.pi))
    print('  V%d =  %7.4f ∠ %8.4f [pu][deg]'%(2, V_All[0], -1*𝜃_All[1]*180/math.pi))
    
    y1 = -(radius**2 - (PG - centreS[0])**2)**0.5 + centreS[1]
    y2 = (radius**2 - (PG - centreS[0])**2)**0.5 + centreS[1]
    print('Consider Q1 = %.4f or Q1 = %.4f'%(y1, y2))
    P = -Results['Net_Power'][0].real/100
    Q = -Results['Net_Power'][0].imag/100
    plot_circles(centreR, centreS, radius, [P, Q], [PG, QG], Axes)

interactive(children=(FloatSlider(value=0.1003, description='P1 pu', max=1.0, step=0.01), FloatSlider(value=0.…

The power circle diagrams and examples above can be used to explore inpacts on voltages from
- Different types of loads, including inductive loads, reactors, capacitors, etc.
- Traditional and distributed generators, condensers, etc.

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)

## Series compensation

The voltage drop across a line can be mitigated by connecting capacitors in series to the lines. This can be explored by simulating the power flows across the lines (using the tools developed in previous notebooks) subject to different line impedances. However, let us also try to visualize the effects of the impedances on the voltages using phasor diagrams.
> Try different values for R and X and check how the voltages change.

In [7]:
@interact
def Phasor_01(R = widgets.FloatSlider(min=-1,max=1,step=0.025, value=0.1, description='R pu'),
         X = widgets.FloatSlider(min=-1,max=1,step=0.025, value=0.3, description='X pu')):
    Connectivity = [
        [1, 2, complex(R, X), 0]
    ]
    Load = [
        [2, complex(0.1, 0.01)]
    ]
    Generator = [
        {'Bus':1, 'V':1, '𝜃':0 }
    ]
    V_All, 𝜃_All, Threshold, Succes = bfs(Generator, Connectivity, Load)
    if not Succes:
        print('The model failed to converge')
    Results = get_Parameters(Connectivity, V_All, 𝜃_All, Succes)
    print('  V%d =  %7.4f ∠ %8.4f [pu][deg]'%(1, V_All[0], 𝜃_All[0]*180/math.pi))
    print('  V%d =  %7.4f ∠ %8.4f [pu][deg]'%(2, V_All[1], 𝜃_All[1]*180/math.pi))
    print('  I  =  %7.4f ∠ %8.4f [pu][deg]'%(abs(Results['Current'][0]),
                                             cmath.phase(Results['Current'][0])*180/math.pi))
    plot_phasor(Results, Connectivity)

interactive(children=(FloatSlider(value=0.1, description='R pu', max=1.0, min=-1.0, step=0.025), FloatSlider(v…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)

## Generators

Synchronous generators can provide voltage support by absorbind or producing reactive power. This is achieved by the automatic voltage regulator (AVR) through adjustments of the field exitation current. A diagram of the generator is presented below.

![Week06_Generator_Phasor.png](Figures/Week06_Generator_Phasor.png)

> Adjust the field exitation current and check how the voltage output of the generator (V) changes

In [8]:
@interact
def Phasor_Gen(Th = widgets.FloatSlider(min=-10,max=10,step=1, value=0, description='Angle I deg')):
    Ik = 0.4
    Ang = Th/180*math.pi
    X = complex(0, 0.5)

    aux = Ik/math.cos(Ang)
    I = complex(Ik, aux*math.sin(Ang))

    V = complex(1, 0)
    IX = -I*X
    E = V-IX

    fig, ax = plt.subplots()
    Origin = [0, 0]
    m, _ = plot_vector(Origin, V, ax, 'V')
    m, Er = plot_vector(Origin, E, ax, 'E')
    m, _ = plot_vector(Er, IX, ax, 'jIX')
    m, _ = plot_vector(Origin, I, ax, 'I')

    ax.set_xlim(-0.05, 1.07)
    ax.set_ylim(-0.1, 0.25)
    plt.grid()
    print('  V =  %7.4f ∠ %8.4f [pu][deg]'%(abs(V), cmath.phase(V)*180/math.pi))
    print('  E =  %7.4f ∠ %8.4f [pu][deg]'%(abs(E), cmath.phase(E)*180/math.pi))
    print('  I =  %7.4f ∠ %8.4f [pu][deg]'%(abs(I), cmath.phase(I)*180/math.pi))
    if Th == 0:
        print('Normal excitation')
    elif Th > 0:
        print('Under-excited')
    else:
        print('Overexcited')

interactive(children=(FloatSlider(value=0.0, description='Angle I deg', max=10.0, min=-10.0, step=1.0), Output…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)

## Tap-changing transformers

Tap-changing transformers can be used to provide voltage support by adjusting their turns ratio through changes to the transformer tap position.

![Week06_Transformer.png](Figures/Week06_Transformer.png)

> Adjust the turns ratio (t) and check how the voltages change.

In [9]:
@interact
def Phasor_Gen(t = widgets.FloatSlider(min=0.85,max=1.05,step=0.01, value=0.9055, description='t')):

    B = 5
    Y = complex(0, B)
    Ybus = [[-t**2*B, t*B],[t*B, -B ]]
    Load = [
        [2, complex(0.5, 0.5)]
    ]
    Generator = [
        {'Bus':1, 'V':.95, '𝜃':0 }
    ]

    P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True, False)
    Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)
    V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, 0)
    print('Vs =  %7.4f ∠ %8.4f [pu][deg]'%(V_All[0], 𝜃_All[0]*180/math.pi))
    print('Vp =  %7.4f ∠ %8.4f [pu][deg]'%(V_All[1], 𝜃_All[1]*180/math.pi))

interactive(children=(FloatSlider(value=0.9055, description='t', max=1.05, min=0.85, step=0.01), Output()), _d…

[Back to top](#EEEN30131-Power-System-Analysis:-Week-06---Examples)